In [8]:
import pandas as pd
import numpy as np
import os
from sklearn.cluster import KMeans
from scipy.stats import norm
import statsmodels.formula.api as smf
import warnings

# --- 1. 风险模型函数 (最终版：适配新M4模型与个体化动态约束) ---

def calculate_r_late(t_days):
    """计算过晚检测风险 R_late"""
    if 70 <= t_days <= 84: return (2/14) * (t_days - 70)
    if 84 < t_days <= 189: return 2 + (4/105) * (t_days - 84)
    if 189 < t_days <= 210: return 0.2225 * np.exp(0.0243 * t_days)
    return np.inf

def calculate_time_value_weight(t_days):
    """计算时间价值权重 W(t)，已乘以0.18"""
    #return 60*(0.01728 + 0.16344 / (1 + np.exp(0.1 * (t_days - 154))))
    return 30
def precompute_valid_start_points(df, fe_params, random_effects):
    """
    为数据集中每一位孕妇预先计算其唯一的、合理的求解起始点（天）。
    采用求导求根的方法确定三次曲线的单调递增区间起点。
    """
    print("    - 步骤A: 为每一位孕妇计算其唯一的曲线导数并求解极值点...")
    start_points = {}
    
    # 提取导数方程所需的系数
    # Y = c3*t^3 + (c2+u1)*t^2 + c1*t + (c0+u0) + ...
    # Y' = 3*c3*t^2 + 2*(c2+u1)*t + c1
    # 我们模型的公式是：Y ~ I(GW**2) + I(GW**3) + I(BMI**3) + GW:I(BMI**2)
    # 随机效应在 I(GW**2)上, 故 Y' = 3*c3*t^2 + 2*(c2+u1)*t + c4*BMI^2
    c3 = fe_params.get("I(孕周 ** 3)", 0)
    c2 = fe_params.get("I(孕周 ** 2)", 0)
    # 交互项 GW:I(BMI**2) 对 t 求导后剩下 I(BMI**2)
    c4_bmi_sq = fe_params.get("孕周:I(孕妇BMI ** 2)", 0)

    for _, row in df.iterrows():
        subject_id = row['孕妇代码']
        bmi = row['孕妇BMI']
        re_subject = random_effects.get(subject_id, pd.Series({'Group': 0, "I(孕周 ** 2)": 0}))
        u1i = re_subject["I(孕周 ** 2)"]
        
        # 构建一元二次方程 Y'= At^2 + Bt + C 的系数
        A = 3 * c3
        B = 2 * (c2 + u1i)
        C = c4_bmi_sq * (bmi**2)
        
        # 解方程 At^2 + Bt + C = 0 找驻点
        roots = np.roots([A, B, C])
        
        # 对于开口向上的三次曲线 (c3 > 0), 单调递增区间从局部最低点开始
        # 局部最低点是两个实根中较大的那个
        if np.isreal(roots).all() and c3 > 0:
            local_minimum_weeks = np.max(roots)
            # 合理的求解起始点，必须晚于该最低点，且不早于10周临床窗口
            valid_start_weeks = max(local_minimum_weeks, 10)
        else:
            # 如果无实根或开口向下，则从10周开始是安全的
            valid_start_weeks = 10
            
        start_points[subject_id] = np.ceil(valid_start_weeks * 7)

    print("    - 步骤B: 所有个体的合理求解起始点计算完成。")
    return start_points

def get_individual_p_fail_constrained(t_days, bmi, fe_params, random_effects_subject, resid_std, valid_start_day):
    """
    计算单个个体的失败概率，严格施加了个体化动态约束。
    """
    if t_days < valid_start_day:
        return 1.0

    t_weeks = t_days / 7.0
    
    # 构建个体化截距和二次项斜率
    individual_intercept = fe_params['Intercept'] + random_effects_subject['Group']
    individual_quad_slope = fe_params["I(孕周 ** 2)"] + random_effects_subject["I(孕周 ** 2)"]
    
    # 使用最终版M4模型公式进行预测
    y_hat_individual = (individual_intercept +
                      individual_quad_slope * (t_weeks**2) +
                      fe_params.get("I(孕周 ** 3)", 0) * (t_weeks**3) +
                      fe_params.get("I(孕妇BMI ** 3)", 0) * (bmi**3) +
                      fe_params.get("孕周:I(孕妇BMI ** 2)", 0) * t_weeks * (bmi**2))
    
    z_score = (0.04 - y_hat_individual) / resid_std
    return norm.cdf(z_score)

def calculate_risk_for_group_constrained(t_days, group_data, fe_params, random_effects, resid_std, start_points_map):
    """为整个分组计算总期望风险，调用带约束的概率函数"""
    r_late_current = calculate_r_late(t_days)
    r_late_future = calculate_r_late(t_days + 14)
    opportunity_cost = r_late_future - r_late_current
    
    individual_p_fails = []
    for _, row in group_data.iterrows():
        subject_id = row['孕妇代码']
        random_effects_subject = random_effects.get(subject_id, pd.Series({'Group': 0, "I(孕周 ** 2)": 0}))
        valid_start_day = start_points_map[subject_id]
        
        p_fail = get_individual_p_fail_constrained(
            t_days, row['孕妇BMI'], fe_params,
            random_effects_subject, resid_std, valid_start_day
        )
        individual_p_fails.append(p_fail)
    
    mean_p_fail = np.mean(individual_p_fails)
    w_t = calculate_time_value_weight(t_days)
    r_fail = mean_p_fail * w_t * opportunity_cost
    
    return r_late_current + r_fail

# --- 主流程 ---
if __name__ == '__main__':
    warnings.filterwarnings("ignore", category=UserWarning)
    INPUT_FILE_PATH = '../../Data/0/男胎_预处理后数据.csv'
    RESULT_DIR = 'Result'
    os.makedirs(RESULT_DIR, exist_ok=True)
    
    print("加载数据...")
    df_for_analysis = pd.read_csv(INPUT_FILE_PATH)
    print(f"加载完成，使用完整的 {len(df_for_analysis)} 行预处理数据集进行分析。")

    print("\n步骤1: 创建用于模型拟合的干净数据集...")
    outlier_cols = ['Y染色体浓度', '孕周', '孕妇BMI']
    masks = [(df_for_analysis[col] < df_for_analysis[col].quantile(0.25) - 1.5 * (df_for_analysis[col].quantile(0.75) - df_for_analysis[col].quantile(0.25))) | 
             (df_for_analysis[col] > df_for_analysis[col].quantile(0.75) + 1.5 * (df_for_analysis[col].quantile(0.75) - df_for_analysis[col].quantile(0.25))) 
             for col in outlier_cols]
    df_for_fitting = df_for_analysis[~pd.concat(masks, axis=1).any(axis=1)].copy()
    print(f"用于模型拟合的干净数据集共 {len(df_for_fitting)} 行。")

    print("\n步骤2: 在干净数据集上拟合最终版混合效应模型...")
    model_formula = "Q('Y染色体浓度') ~ I(孕周**2) + I(孕周**3) + I(孕妇BMI**3) + 孕周:I(孕妇BMI**2)"
    mixed_model = smf.mixedlm(model_formula, df_for_fitting, groups=df_for_fitting["孕妇代码"], re_formula="~ I(孕周**2)")
    results = mixed_model.fit()
    print("模型拟合完成。")
    
    fe_params = results.fe_params
    random_effects = results.random_effects
    resid_std = np.sqrt(results.scale)
    
    print("\n步骤3: 预计算所有个体的动态约束条件 (求导求根)...")
    start_points_map = precompute_valid_start_points(df_for_analysis, fe_params, random_effects)
    
    print("\n--- 开始执行双层优化流程 (已应用最终版模型与约束) ---")
    t_grid = np.arange(70, 197)
    all_results = []
    k_risks = {}
    K_RANGE = range(2, 9)

    for k in K_RANGE:
        print(f"\n------ 正在处理 K = {k} 的情况 ---")
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        df_for_analysis['cluster'] = kmeans.fit_predict(df_for_analysis[['孕妇BMI']])
        
        total_weighted_risk = 0
        
        for i in range(k):
            group_df = df_for_analysis[df_for_analysis['cluster'] == i]
            
            risks = [calculate_risk_for_group_constrained(t, group_df, fe_params, random_effects, resid_std, start_points_map) for t in t_grid]
            
            min_risk_index = np.argmin(risks)
            best_t = t_grid[min_risk_index]
            min_risk = risks[min_risk_index]
            
            total_weighted_risk += min_risk * len(group_df)
            
            result_entry = {
                'K': k, '群组ID': i, '孕妇数量': len(group_df),
                'BMI范围': f"[{group_df['孕妇BMI'].min():.2f}, {group_df['孕妇BMI'].max():.2f}]",
                '平均BMI': group_df['孕妇BMI'].mean(),
                '最佳时点_天': best_t,
                '最小风险': min_risk
            }
            all_results.append(result_entry)
            print(f"  - 分组 {i}: N={len(group_df)}, BMI范围=[{group_df['孕妇BMI'].min():.2f}, {group_df['孕妇BMI'].max():.2f}], "
                  f"最优时点 t*={best_t} 天 (约{best_t/7:.1f}周), 最小期望风险={min_risk:.4f}")

        k_risks[k] = total_weighted_risk / len(df_for_analysis)
        print(f"--- K = {k} 的加权平均总风险为: {k_risks[k]:.6f} ---")

    best_k = min(k_risks, key=k_risks.get)
    print(f"\n--- 优化流程结束 ---\n最优分组数 K* = {best_k}，其加权平均总风险最低，为 {k_risks[best_k]:.6f}")

    all_results_df = pd.DataFrame(all_results)
    best_k_results_df = all_results_df[all_results_df['K'] == best_k].copy()
    
    bmi_means = best_k_results_df.sort_values('平均BMI')['群组ID'].values
    name_map = {bmi_means[i]: f'群组{i+1}' for i, _ in enumerate(bmi_means)}
    best_k_results_df['群组名称'] = best_k_results_df['群组ID'].map(name_map)
    best_k_results_df['最佳时点_周'] = best_k_results_df['最佳时点_天'] / 7
    
    final_cols = ['群组名称', 'BMI范围', '孕妇数量', '平均BMI', '最佳时点_天', '最佳时点_周', '最小风险']
    print("\n最终最优方案详情:")
    print(best_k_results_df.sort_values('平均BMI')[final_cols].to_string(index=False))

    best_k_results_df.to_csv(os.path.join(RESULT_DIR, '问题二_最终分组优化方案.csv'), index=False, encoding='utf-8-sig')
    k_risks_df = pd.DataFrame(list(k_risks.items()), columns=['K值', '加权平均总风险'])
    k_risks_df.to_csv(os.path.join(RESULT_DIR, '问题二_不同K值总风险对比.csv'), index=False, encoding='utf-8-sig')

    kmeans = KMeans(n_clusters=best_k, random_state=42, n_init=10)
    df_for_analysis['cluster'] = kmeans.fit_predict(df_for_analysis[['孕妇BMI']])
    df_for_analysis.to_csv(os.path.join(RESULT_DIR, '问题二_带最优分组标签的数据.csv'), index=False, encoding='utf-8-sig')

    risk_curves_data = []
    for i in range(best_k):
        group_df = df_for_analysis[df_for_analysis['cluster'] == i]
        group_name = name_map[i]
        for t in t_grid:
            risk = calculate_risk_for_group_constrained(t, group_df, fe_params, random_effects, resid_std, start_points_map)
            risk_curves_data.append({'K': best_k, '群组名称': group_name, '天数': t, '期望风险': risk})
    
    pd.DataFrame(risk_curves_data).to_csv(os.path.join(RESULT_DIR, '问题二_最优方案风险曲线数据.csv'), index=False, encoding='utf-8-sig')
    
    print("\n所有计算与数据保存已完成。")

加载数据...
加载完成，使用完整的 998 行预处理数据集进行分析。

步骤1: 创建用于模型拟合的干净数据集...
用于模型拟合的干净数据集共 962 行。

步骤2: 在干净数据集上拟合最终版混合效应模型...
模型拟合完成。

步骤3: 预计算所有个体的动态约束条件 (求导求根)...
    - 步骤A: 为每一位孕妇计算其唯一的曲线导数并求解极值点...
    - 步骤B: 所有个体的合理求解起始点计算完成。

--- 开始执行双层优化流程 (已应用最终版模型与约束) ---

------ 正在处理 K = 2 的情况 ---
  - 分组 0: N=362, BMI范围=[32.94, 46.88], 最优时点 t*=147 天 (约21.0周), 最小期望风险=5.7300
  - 分组 1: N=636, BMI范围=[20.70, 32.93], 最优时点 t*=129 天 (约18.4周), 最小期望风险=5.1793
--- K = 2 的加权平均总风险为: 5.379057 ---

------ 正在处理 K = 3 的情况 ---
  - 分组 0: N=416, BMI范围=[31.53, 35.44], 最优时点 t*=130 天 (约18.6周), 最小期望风险=5.2102
  - 分组 1: N=132, BMI范围=[35.49, 46.88], 最优时点 t*=173 天 (约24.7周), 最小期望风险=6.3376
  - 分组 2: N=450, BMI范围=[20.70, 31.48], 最优时点 t*=129 天 (约18.4周), 最小期望风险=5.1437
--- K = 3 的加权平均总风险为: 5.329343 ---

------ 正在处理 K = 4 的情况 ---
  - 分组 0: N=383, BMI范围=[20.70, 31.00], 最优时点 t*=146 天 (约20.9周), 最小期望风险=5.2440
  - 分组 1: N=372, BMI范围=[31.02, 33.89], 最优时点 t*=130 天 (约18.6周), 最小期望风险=5.2321
  - 分组 2: N=43, BMI范围=[38.22, 46.88], 最优时点 t*=165 天 (约23.6周), 最小

In [3]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import statsmodels.formula.api as smf
import warnings

# --- 1. 绘图风格与中文支持设置 ---
def setup_plot_style():
    """设置图表风格和中文支持"""
    plt.rcParams['font.sans-serif'] = ['SimHei']
    plt.rcParams['axes.unicode_minus'] = False
    academic_colors = {
        'blue': '#0c5da5', 'teal': '#00b9a9', 'orange': '#f7931e',
        'red': '#d52b1e', 'purple': '#8f5b9f', 'gray': '#808080'
    }
    plt.style.use('default')
    plt.rcParams.update({
        'axes.grid': True,
        'grid.color': '#cccccc',
        'grid.linestyle': '--',
        'grid.linewidth': 0.5,
        'axes.edgecolor': 'black',
        'axes.linewidth': 1.5,
        'axes.labelsize': 20,
        'axes.titlesize': 22,
        'xtick.labelsize': 18,
        'ytick.labelsize': 18,
        'legend.fontsize': 16,
        'figure.figsize': [14, 9],
        'figure.dpi': 300,
    })
    return academic_colors

# --- 2. 主分析与绘图流程 ---
if __name__ == '__main__':
    warnings.filterwarnings("ignore", category=UserWarning)
    # --- 数据加载 ---
    INPUT_FILE_PATH = '../../Data/0/男胎_预处理后数据.csv'
    RESULT_DIR = 'Result'
    os.makedirs(RESULT_DIR, exist_ok=True)
    
    print("加载数据...")
    df_for_analysis = pd.read_csv(INPUT_FILE_PATH)
    print(f"加载完成，使用完整的 {len(df_for_analysis)} 行预处理数据集进行分析。")

    # --- 步骤1: 创建用于模型拟合的干净数据集 (已修正) ---
    # 不再读取外部文件，而是动态执行与第一问日志一致的筛选
    print("\n步骤1: 创建用于模型拟合的干净数据集...")
    outlier_cols = ['Y染色体浓度', '孕周', '孕妇BMI']
    
    masks = []
    for col in outlier_cols:
        Q1 = df_for_analysis[col].quantile(0.25)
        Q3 = df_for_analysis[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        mask = (df_for_analysis[col] < lower_bound) | (df_for_analysis[col] > upper_bound)
        masks.append(mask)
        
    combined_outliers = pd.concat(masks, axis=1).any(axis=1)
    df_for_fitting = df_for_analysis[~combined_outliers].copy()
    print(f"用于模型拟合的干净数据集共 {len(df_for_fitting)} 行。")

    # --- 后续步骤不变 ---
    print("\n步骤2: 在干净数据集上拟合最终版混合效应模型...")
    model_formula = "Q('Y染色体浓度') ~ I(孕周**2) + I(孕周**3) + I(孕妇BMI**3) + 孕周:I(孕妇BMI**2)"
    mixed_model = smf.mixedlm(model_formula, df_for_fitting, groups=df_for_fitting["孕妇代码"], re_formula="~ I(孕周**2)")
    results = mixed_model.fit()
    print("模型拟合完成。")
    
    fe_params = results.fe_params
    random_effects = results.random_effects
    resid_std = np.sqrt(results.scale)
    
    print("\n步骤3: 从完整数据集中随机抽取30位孕妇...")
    sampled_individuals = df_for_analysis.sample(n=30, random_state=42)
    print("抽样完成。")

    print("\n步骤4: 为抽样个体计算唯一的概率演化曲线...")
    t_weeks = np.linspace(5, 28, 200)
    all_curves = []

    for _, row in sampled_individuals.iterrows():
        subject_id = row['孕妇代码']
        bmi = row['孕妇BMI']
        random_effects_subject = random_effects.get(subject_id, pd.Series({'Group': 0, "I(孕周 ** 2)": 0}))
        
        individual_intercept = fe_params['Intercept'] + random_effects_subject['Group']
        individual_quad_slope = fe_params["I(孕周 ** 2)"] + random_effects_subject["I(孕周 ** 2)"]
        
        y_hat_individual = (individual_intercept +
                          individual_quad_slope * (t_weeks**2) +
                          fe_params.get("I(孕周 ** 3)", 0) * (t_weeks**3) +
                          fe_params.get("I(孕妇BMI ** 3)", 0) * (bmi**3) +
                          fe_params.get("孕周:I(孕妇BMI ** 2)", 0) * t_weeks * (bmi**2))
    
        z_score = (0.04 - y_hat_individual) / resid_std
        prob_curve = 1 - norm.cdf(z_score)
        all_curves.append(prob_curve)
    print("所有个体曲线计算完成。")
    
    print("\n步骤5: 正在生成可视化图表...")
    colors = setup_plot_style()
    fig, ax = plt.subplots()

    for curve in all_curves:
        ax.plot(t_weeks, curve, color=colors['blue'], lw=1.0, alpha=0.35)
    
    mean_curve = np.mean(all_curves, axis=0)
    ax.plot(t_weeks, mean_curve, color=colors['red'], lw=3.5, linestyle='-', label='30个样本平均趋势')

    ax.axhline(y=0.95, color='black', linestyle='--', lw=2.0, label='95% 可靠性阈值')
    ax.axvline(x=10, color=colors['gray'], linestyle=':', lw=2.0, label='10周 (临床窗口起点)')

    ax.set_xlabel('检测孕周')
    ax.set_ylabel('Y染色体浓度达标概率')
    ax.legend()

    ax.yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1.0))
    ax.xaxis.set_major_locator(mticker.MultipleLocator(4))
    ax.xaxis.set_minor_locator(mticker.MultipleLocator(1))
    
    ax.set_xlim(5, 28)
    ax.set_ylim(0, 1.05)
    
    plt.tight_layout()
    
    SAVE_PATH = os.path.join(RESULT_DIR, '图9_最终M4模型个体概率曲线.png')
    plt.savefig(SAVE_PATH)
    plt.close()
    
    print(f"\n分析完成，图表已保存至: {SAVE_PATH}")

加载数据...
加载完成，使用完整的 998 行预处理数据集进行分析。

步骤1: 创建用于模型拟合的干净数据集...
用于模型拟合的干净数据集共 962 行。

步骤2: 在干净数据集上拟合最终版混合效应模型...
模型拟合完成。

步骤3: 从完整数据集中随机抽取30位孕妇...
抽样完成。

步骤4: 为抽样个体计算唯一的概率演化曲线...
所有个体曲线计算完成。

步骤5: 正在生成可视化图表...

分析完成，图表已保存至: Result\图9_最终M4模型个体概率曲线.png
